# Introdução à Agronomia com Python e dados (MC 5)

### Professor: Lucas Mielke.
### Contato: lucas.mielke@usp.br

## Tópico 2.4  Dados municipais (IBGE) (consulta de dados online). 

API - "Ponte de dados", segura e controlável.

API na prática - endereço padronizado (query) e que exibe dados estruturados (em json, cvs..).

As vezes temos bibliotecas próprias (Quandl), As vezes não (IBGE) - quando não temos, usamos ferramentas de leitura. 


Exemplo: https://www.quandl.com/data/CEPEA/SUGAR_C-Brazilian-Agribusiness-Price-Indices-Crystal-Sugar-Cash

Os API Json e CSV podem ser lidos diretamente do navegador, assim como pelo "GET" do Python ou Pandas

In [1]:
import pandas as pd
sexo = "f"
pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=20&sexo="+sexo)

,nome,regiao,freq,rank,sexo
0,MARIA,0,11694738,1,F
1,ANA,0,3079729,2,F
2,FRANCISCA,0,721637,3,F
3,ANTONIA,0,588783,4,F
4,ADRIANA,0,565621,5,F
5,JULIANA,0,562589,6,F
6,MARCIA,0,551855,7,F
7,FERNANDA,0,531607,8,F
8,PATRICIA,0,529446,9,F
9,ALINE,0,509869,10,F


Muitos locais armazenam API em json. O json agrega informações de uma maneira simples e legível, entretanto a leitura começa a ficar complicada se ele tiver vários níveis (chamados de **nested json**), como no exemplo abaixo:

In [3]:
a = pd.read_json("http://servicodados.ibge.gov.br/api/v2/metadados/pesquisas")
a

,codigo,nome,nome_ingles,situacao,categoria,periodicidade_coleta,periodicidade_divulgacao,classificacoes_tematicas
0,AC,Pesquisa Anual da Indústria da Construção,None,Ativa,Estrutural,Anual,Anual,"[{'nome': 'Estatísticas econômicas setoriais',..."
1,AA,Pesquisa Nacional de Saúde do Escolar,None,Ativa,Especial,Eventual,Eventual,"[{'nome': 'Grupos populacionais específicos', ..."
2,AM,Pesquisa de Assistência Médico-Sanitária,None,Ativa,Especial,Eventual,Eventual,"[{'nome': 'Saúde', 'nome_ingles': 'Health', 'd..."
3,AX,Pesquisa Trimestral do Abate de Animais,Quarterly Survey of Animal Slaughter,Ativa,Conjuntural,Trimestral,Trimestral,"[{'nome': 'Estatísticas econômicas setoriais',..."
4,CD,Censo Demográfico,Population and Housing Census,Ativa,Estrutural,Decenal,Decenal,"[{'nome': 'Família', 'nome_ingles': 'Household..."
...,...,...,...,...,...,...,...,...
80,DT,Contas de Espécies Ameaçadas,Threatened Species Accounts,Ativa,Especial,Não se aplica,Quinquenal,"[{'nome': 'Meio Ambiente', 'nome_ingles': 'Env..."
81,DO,Panorama Nacional e Internacional da Produção ...,None,Concluída,Especial,Não se aplica,Eventual,"[{'nome': 'Família', 'nome_ingles': 'Household..."
82,PZ,Pesquisa Industrial Mensal - Produção Física,None,Ativa,Conjuntural,Mensal,Mensal,"[{'nome': 'Indústrias extrativas', 'nome_ingle..."
83,LA,Levantamento Sistemático da Produção Agrícola,Systematic Survey of Agricultural Production,Ativa,Conjuntural,Mensal,Mensal,"[{'nome': 'Estatísticas econômicas setoriais',..."


In [8]:
dicionario = a["classificacoes_tematicas"][0][0]
dicionario.keys()

dict_keys(['nome', 'nome_ingles', 'descricao', 'descricao_ingles', 'dominio', 'dominio_ingles'])

Nesses casos é necessário normalizar o Json ou achatá-lo (flatten). Vamos fazer juntos uma função API do IBGE onde vamos tratar esse tema

## Explorando API do IBGE

https://servicodados.ibge.gov.br/api/docs - todos APIS

https://servicodados.ibge.gov.br/api/docs/agregados?versao=3 - APIS agregados

O API é amigável e possui um Query builder

https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/2019/variaveis/216?localidades=N9[11001]&classificacao=782[40129]

onde:

    5417 = Relatório de produção

    2019 = ano (2019)

    216 = tipo de relatorio (área colhida)

    11001 = microregião/cidade (porto velho)
    
    N9 = tipo de região analisada

    40129 = cultura (abacate)

In [13]:
ano="2019"
var="216"
cidade="11001"
cultura="40129"
geo="9"
    
url = "https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/"+ano+"/variaveis/"+var+"?localidades=N"+geo+"["+cidade+"]&classificacao=782["+cultura+"]"
print(url)

https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/2019/variaveis/216?localidades=N9[11001]&classificacao=782[40129]


In [18]:
import requests
API = requests.get(url)
print(API,"\n")
dic = API.json()
print("Voilá, temos o API:\n\n",dic)


<Response [200]> 

Voilá, temos o API:

 [{'id': '216', 'variavel': 'Área colhida', 'unidade': 'Hectares', 'resultados': [{'classificacoes': [{'id': '782', 'nome': 'Produto das lavouras temporárias e permanentes', 'categoria': {'40129': 'Abacate'}}], 'series': [{'localidade': {'id': '11001', 'nivel': {'id': 'N9', 'nome': 'Microrregião Geográfica'}, 'nome': 'Porto Velho (RO)'}, 'serie': {'2019': '-'}}]}]}]


http://jsonviewer.stack.hu/

## Vamos arruamar esse arquivo usando o Normalize
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html

In [19]:
import pandas as pd

from pandas.io.json import json_normalize

In [21]:
dic

[{'id': '216',
  'variavel': 'Área colhida',
  'unidade': 'Hectares',
  'resultados': [{'classificacoes': [{'id': '782',
      'nome': 'Produto das lavouras temporárias e permanentes',
      'categoria': {'40129': 'Abacate'}}],
    'series': [{'localidade': {'id': '11001',
       'nivel': {'id': 'N9', 'nome': 'Microrregião Geográfica'},
       'nome': 'Porto Velho (RO)'},
      'serie': {'2019': '-'}}]}]}]

In [22]:
pd.json_normalize(dic)

,id,variavel,unidade,resultados
0,216,Área colhida,Hectares,"[{'classificacoes': [{'id': '782', 'nome': 'Pr..."


In [23]:
pd.json_normalize(dic,record_path=['resultados'],meta=['variavel','unidade',"id"]) #usando normalize pela primeira vez

,classificacoes,series,variavel,unidade,id
0,"[{'id': '782', 'nome': 'Produto das lavouras t...","[{'localidade': {'id': '11001', 'nivel': {'id'...",Área colhida,Hectares,216


In [25]:
pd.json_normalize(dic,record_path=['resultados','series'],meta=['variavel','unidade',"id"]) #usando normalize pela primeira vez 

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2019,variavel,unidade,id
0,11001,N9,Microrregião Geográfica,Porto Velho (RO),-,Área colhida,Hectares,216


In [26]:
pd.json_normalize(dic,record_path=['resultados','classificacoes'],meta=['variavel','unidade']) #usando normalize pela segunda vez


,id,nome,categoria.40129,variavel,unidade
0,782,Produto das lavouras temporárias e permanentes,Abacate,Área colhida,Hectares


In [29]:
dados = pd.json_normalize(dic,record_path=['resultados','series'],meta=['variavel','unidade',"id"]) #usando normalize pela primeira vez 
dados2 = pd.json_normalize(dic,record_path=['resultados','classificacoes'],meta=['variavel','unidade']) 
dados3 = dados.merge(dados2, left_index=True, right_index=True)
dados3 

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2019,variavel_x,unidade_x,id_x,id_y,nome,categoria.40129,variavel_y,unidade_y
0,11001,N9,Microrregião Geográfica,Porto Velho (RO),-,Área colhida,Hectares,216,782,Produto das lavouras temporárias e permanentes,Abacate,Área colhida,Hectares


## Vamos arruamar esse arquivo usando o Flatten
https://medium.com/@amirziai/flatten-json-on-python-package-index-pypi-9e4951693a5a

In [30]:
dic

[{'id': '216',
  'variavel': 'Área colhida',
  'unidade': 'Hectares',
  'resultados': [{'classificacoes': [{'id': '782',
      'nome': 'Produto das lavouras temporárias e permanentes',
      'categoria': {'40129': 'Abacate'}}],
    'series': [{'localidade': {'id': '11001',
       'nivel': {'id': 'N9', 'nome': 'Microrregião Geográfica'},
       'nome': 'Porto Velho (RO)'},
      'serie': {'2019': '-'}}]}]}]

In [31]:
!pip install flatten_json
from flatten_json import flatten

O json é quase uma lista de dicionários... se pegarmos o primeiro termo usando [0]

In [32]:
flatten(dic[0]) #o primeiro elemento do json é um dicionário

{'id': '216',
 'variavel': 'Área colhida',
 'unidade': 'Hectares',
 'resultados_0_classificacoes_0_id': '782',
 'resultados_0_classificacoes_0_nome': 'Produto das lavouras temporárias e permanentes',
 'resultados_0_classificacoes_0_categoria_40129': 'Abacate',
 'resultados_0_series_0_localidade_id': '11001',
 'resultados_0_series_0_localidade_nivel_id': 'N9',
 'resultados_0_series_0_localidade_nivel_nome': 'Microrregião Geográfica',
 'resultados_0_series_0_localidade_nome': 'Porto Velho (RO)',
 'resultados_0_series_0_serie_2019': '-'}

Também podemos pegar cada termo do dicionário usando for e aplicar o flatten

In [33]:
dic_flattened = [flatten(d) for d in dic]
df = pd.DataFrame(dic_flattened)
df

,id,variavel,unidade,resultados_0_classificacoes_0_id,resultados_0_classificacoes_0_nome,resultados_0_classificacoes_0_categoria_40129,resultados_0_series_0_localidade_id,resultados_0_series_0_localidade_nivel_id,resultados_0_series_0_localidade_nivel_nome,resultados_0_series_0_localidade_nome,resultados_0_series_0_serie_2019
0,216,Área colhida,Hectares,782,Produto das lavouras temporárias e permanentes,Abacate,11001,N9,Microrregião Geográfica,Porto Velho (RO),-


Ficou praticamente identico, porém o resultado do segundo método seria ruim para pesquisas mais completas, pois teríamos datasets com centenas de colunas

### Fazendo funções

**lembrando que:**:

https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/2019/variaveis/216?localidades=N9[11001]&classificacao=782[40129]

onde:

    5417 = Relatório de produção

    2019 = ano (2019)

    216 = tipo de variável relatorio (área colhida)

    11001 = microregião/cidade (porto velho)
    
    N9 = tipo de região analisada

    40129 = cultura (abacate)

In [34]:
#Usando normalize
import pandas as pd
from pandas.io.json import json_normalize
import requests

def ibge_pam(ano="2019",var="216",cidade="11001",cultura="40129",geo="9"):
    #busca a pesquisa agricola municipal
    url = "https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/"+ano+"/variaveis/"+var+"?localidades=N"+geo+"["+cidade+"]&classificacao=782["+cultura+"]"
    dataAPI = requests.get(url) #request get do api - url formado acima
    dic = dataAPI.json() #convertendo em json
    dados = pd.json_normalize(dic,record_path=['resultados','series'],meta=['variavel','unidade',"id"]) #usando normalize pela primeira vez 
    dados2 = pd.json_normalize(dic,record_path=['resultados','classificacoes'],meta=['variavel','unidade']) 
    dados3 = dados.merge(dados2, left_index=True, right_index=True)
    return dados3   

In [35]:
#usando Flatten
import pandas as pd
from pandas.io.json import json_normalize
import requests
from flatten_json import flatten

def ibge_pam2(ano="2019",var="216",cidade="11001",cultura="40129",geo="9"):
    #busca a pesquisa agricola municipal
    url = "https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/"+ano+"/variaveis/"+var+"?localidades=N"+geo+"["+cidade+"]&classificacao=782["+cultura+"]"
    dataAPI = requests.get(url) #request get do api - url formado acima
    dic = dataAPI.json() #convertendo em json

    dic_flattened = [flatten(d) for d in dic]
    dic_flattened

    df = pd.DataFrame(dic_flattened)
    return df

Codigos das municípios: https://www.ibge.gov.br/explica/codigos-dos-municipios.php

In [40]:
ibge_pam(ano="2019",var="216",cidade="11001",cultura="40124",geo="9")

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2019,variavel_x,unidade_x,id_x,id_y,nome,categoria.40124,variavel_y,unidade_y
0,11001,N9,Microrregião Geográfica,Porto Velho (RO),36331,Área colhida,Hectares,216,782,Produto das lavouras temporárias e permanentes,Soja (em grão),Área colhida,Hectares


In [41]:
ibge_pam2(ano="2019",var="216",cidade="11001",cultura="40124",geo="9")

,id,variavel,unidade,resultados_0_classificacoes_0_id,resultados_0_classificacoes_0_nome,resultados_0_classificacoes_0_categoria_40124,resultados_0_series_0_localidade_id,resultados_0_series_0_localidade_nivel_id,resultados_0_series_0_localidade_nivel_nome,resultados_0_series_0_localidade_nome,resultados_0_series_0_serie_2019
0,216,Área colhida,Hectares,782,Produto das lavouras temporárias e permanentes,Soja (em grão),11001,N9,Microrregião Geográfica,Porto Velho (RO),36331


## Vamos fazer um json mais complexo, pesquisando todas culturas agrícolas

Vamos repetir as etapas da técnica do normalized

https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/2019/variaveis/216%7C214%7C215?localidades=N1[1]&classificacao=782[all]

In [42]:
ano="2019"
rel="216"
loc="1"
geo="1"

In [43]:
url = "https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/"+ano+"/variaveis/"+rel+"%7C214%7C215?localidades=N"+geo+"["+loc+"]&classificacao=782[all]"
print(url)

https://servicodados.ibge.gov.br/api/v3/agregados/5457/periodos/2019/variaveis/216%7C214%7C215?localidades=N1[1]&classificacao=782[all]


In [44]:
import pandas as pd

from pandas.io.json import json_normalize

import requests

dataAPI = requests.get(url)

print(dataAPI.json()) 

[{'id': '216', 'variavel': 'Área colhida', 'unidade': 'Hectares', 'resultados': [{'classificacoes': [{'id': '782', 'nome': 'Produto das lavouras temporárias e permanentes', 'categoria': {'0': 'Total'}}], 'series': [{'localidade': {'id': '1', 'nivel': {'id': 'N1', 'nome': 'Brasil'}, 'nome': 'Brasil'}, 'serie': {'2019': '80576400'}}]}, {'classificacoes': [{'id': '782', 'nome': 'Produto das lavouras temporárias e permanentes', 'categoria': {'40129': 'Abacate'}}], 'series': [{'localidade': {'id': '1', 'nivel': {'id': 'N1', 'nome': 'Brasil'}, 'nome': 'Brasil'}, 'serie': {'2019': '15315'}}]}, {'classificacoes': [{'id': '782', 'nome': 'Produto das lavouras temporárias e permanentes', 'categoria': {'40092': 'Abacaxi'}}], 'series': [{'localidade': {'id': '1', 'nivel': {'id': 'N1', 'nome': 'Brasil'}, 'nome': 'Brasil'}, 'serie': {'2019': '67167'}}]}, {'classificacoes': [{'id': '782', 'nome': 'Produto das lavouras temporárias e permanentes', 'categoria': {'45982': 'Açaí'}}], 'series': [{'localidad

In [45]:
dic = dataAPI.json()
dic

[{'id': '216',
  'variavel': 'Área colhida',
  'unidade': 'Hectares',
  'resultados': [{'classificacoes': [{'id': '782',
      'nome': 'Produto das lavouras temporárias e permanentes',
      'categoria': {'0': 'Total'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'2019': '80576400'}}]},
   {'classificacoes': [{'id': '782',
      'nome': 'Produto das lavouras temporárias e permanentes',
      'categoria': {'40129': 'Abacate'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'2019': '15315'}}]},
   {'classificacoes': [{'id': '782',
      'nome': 'Produto das lavouras temporárias e permanentes',
      'categoria': {'40092': 'Abacaxi'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'2019': '67167'}}]},
   {'classificacoes': [{'id'

In [46]:
pd.json_normalize(dic)

,id,variavel,unidade,resultados
0,216,Área colhida,Hectares,"[{'classificacoes': [{'id': '782', 'nome': 'Pr..."
1,214,Quantidade produzida,Toneladas,"[{'classificacoes': [{'id': '782', 'nome': 'Pr..."
2,215,Valor da produção,Mil Reais,"[{'classificacoes': [{'id': '782', 'nome': 'Pr..."


In [47]:
dados = pd.json_normalize(dic,record_path=['resultados'],meta=['variavel','unidade',"id"])
dados.head(3)

,classificacoes,series,variavel,unidade,id
0,"[{'id': '782', 'nome': 'Produto das lavouras t...","[{'localidade': {'id': '1', 'nivel': {'id': 'N...",Área colhida,Hectares,216
1,"[{'id': '782', 'nome': 'Produto das lavouras t...","[{'localidade': {'id': '1', 'nivel': {'id': 'N...",Área colhida,Hectares,216
2,"[{'id': '782', 'nome': 'Produto das lavouras t...","[{'localidade': {'id': '1', 'nivel': {'id': 'N...",Área colhida,Hectares,216


In [48]:
dados = pd.json_normalize(dic,record_path=['resultados','series'],meta=['variavel','unidade',"id"])
dados.head(3)

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2019,variavel,unidade,id
0,1,N1,Brasil,Brasil,80576400,Área colhida,Hectares,216
1,1,N1,Brasil,Brasil,15315,Área colhida,Hectares,216
2,1,N1,Brasil,Brasil,67167,Área colhida,Hectares,216


In [49]:
dados2 = pd.json_normalize(dic,record_path=['resultados','classificacoes'],meta=['variavel','unidade'])
dados2.head(3)

,id,nome,categoria.0,categoria.40129,categoria.40092,categoria.45982,categoria.40329,categoria.40130,categoria.40099,categoria.40100,...,categoria.40125,categoria.40271,categoria.40126,categoria.40127,categoria.40128,categoria.40272,categoria.40273,categoria.40274,variavel,unidade
0,782,Produto das lavouras temporárias e permanentes,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares
1,782,Produto das lavouras temporárias e permanentes,NaN,Abacate,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares
2,782,Produto das lavouras temporárias e permanentes,NaN,NaN,Abacaxi,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares


In [50]:
dados3 = dados.merge(dados2, left_index=True, right_index=True)
dados3.head(3)

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2019,variavel_x,unidade_x,id_x,id_y,nome,...,categoria.40125,categoria.40271,categoria.40126,categoria.40127,categoria.40128,categoria.40272,categoria.40273,categoria.40274,variavel_y,unidade_y
0,1,N1,Brasil,Brasil,80576400,Área colhida,Hectares,216,782,Produto das lavouras temporárias e permanentes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares
1,1,N1,Brasil,Brasil,15315,Área colhida,Hectares,216,782,Produto das lavouras temporárias e permanentes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares
2,1,N1,Brasil,Brasil,67167,Área colhida,Hectares,216,782,Produto das lavouras temporárias e permanentes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Área colhida,Hectares


Olha quantas colunas! Precisamos resolver isso

In [51]:
dados3.columns

Index(['localidade.id', 'localidade.nivel.id', 'localidade.nivel.nome',
       'localidade.nome', 'serie.2019', 'variavel_x', 'unidade_x', 'id_x',
       'id_y', 'nome', 'categoria.0', 'categoria.40129', 'categoria.40092',
       'categoria.45982', 'categoria.40329', 'categoria.40130',
       'categoria.40099', 'categoria.40100', 'categoria.40101',
       'categoria.40102', 'categoria.40103', 'categoria.40131',
       'categoria.40136', 'categoria.40104', 'categoria.40105',
       'categoria.40137', 'categoria.40468', 'categoria.40138',
       'categoria.40139', 'categoria.40140', 'categoria.40141',
       'categoria.40330', 'categoria.40106', 'categoria.40331',
       'categoria.40142', 'categoria.40143', 'categoria.40107',
       'categoria.40108', 'categoria.40109', 'categoria.40144',
       'categoria.40145', 'categoria.40146', 'categoria.40147',
       'categoria.40110', 'categoria.40111', 'categoria.40112',
       'categoria.40148', 'categoria.40113', 'categoria.40114',
       'c

In [52]:
dados3.drop(columns=['localidade.id', 'localidade.nivel.id', 'localidade.nivel.nome','id_x','id_y','variavel_y',
       'unidade_y'],inplace=True) #removo essas colunas
dados3.rename(columns={'serie.'+ano: 'valor', 'variavel_x':'variavel', 'unidade_x':'unidade','localidade.nome':'localidade'},inplace=True)
#renomeio as colunas acima, inclusive a "série.2019" (serie.+ano)

In [57]:
dados3.head(3)

,localidade,valor,variavel,unidade,nome,categoria.0,categoria.40129,categoria.40092,categoria.45982,categoria.40329,...,categoria.40124,categoria.40125,categoria.40271,categoria.40126,categoria.40127,categoria.40128,categoria.40272,categoria.40273,categoria.40274,Categoria
0,Brasil,80576400,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Total,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total
1,Brasil,15315,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,NaN,Abacate,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abacate
2,Brasil,67167,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,NaN,NaN,Abacaxi,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abacaxi


Vou combinar todas as colunas "categoria" usando a tática abaixo

In [58]:
#https://stackoverflow.com/questions/33098383/merge-multiple-column-values-into-one-column-in-python-pandas
dados3['Categoria'] = dados3.filter(like="categoria").apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
dados3['Categoria']    

0                    Total
1                  Abacate
2                  Abacaxi
3                     Açaí
4            Alfafa fenada
              ...         
211        Trigo (em grão)
212    Triticale (em grão)
213    Tungue (fruto seco)
214       Urucum (semente)
215                    Uva
Name: Categoria, Length: 216, dtype: object

Deletando todas as colunas "categoria.XXXX"

In [59]:
for coluna in dados3.columns:
    if coluna[:10] =="categoria.":
        dados3.drop(columns=[coluna],inplace=True)

In [60]:
dados3.shape

(216, 6)

temos 216 linhas de dados estruturados

In [61]:
dados3.head(3)

,localidade,valor,variavel,unidade,nome,Categoria
0,Brasil,80576400,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Total
1,Brasil,15315,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacate
2,Brasil,67167,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacaxi


Se fossemos usar flatten, teríamos 579 colunas

In [62]:
dic_flattened = [flatten(d) for d in dic]
df = pd.DataFrame(dic_flattened)
df.shape

(3, 579)

https://www.ibge.gov.br/explica/codigos-dos-municipios.php

#### Para encerrar, vamos fazer uma função final

In [63]:
import pandas as pd
from pandas.io.json import json_normalize
import requests

In [66]:
def igbe_pam_total(ano="2019",var="216",loc="1",geo="1",rel="5457"):
    
    import pandas as pd
    from pandas.io.json import json_normalize
    import requests
    
    url = "https://servicodados.ibge.gov.br/api/v3/agregados/"+rel+"/periodos/"+ano+"/variaveis/"+var+"%7C214%7C215?localidades=N"+geo+"["+loc+"]&classificacao=782[all]"
    dataAPI = requests.get(url) #request get do api - url formado acima
    dic = dataAPI.json() #convertendo em json
    dados = pd.json_normalize(dic,record_path=['resultados','series'],meta=['variavel','unidade',"id"]) #usando normalize pela primeira vez 
    dados2 = pd.json_normalize(dic,record_path=['resultados','classificacoes'],meta=['variavel','unidade']) 
    dados3 = dados.merge(dados2, left_index=True, right_index=True)
    
    del dataAPI
    del dic
    del dados
    del dados2
    
    dados3.drop(columns=['localidade.id', 'localidade.nivel.id', 'localidade.nivel.nome','id_x','id_y','variavel_y',
       'unidade_y'],inplace=True) #removo essas colunas
    dados3.rename(columns={'serie.'+ano: 'valor', 'variavel_x':'variavel', 'unidade_x':'unidade','localidade.nome':'localidade'},inplace=True)
    #renomeio as colunas acima, inclusive a "série.2019" (serie.+ano)
    
    #https://stackoverflow.com/questions/33098383/merge-multiple-column-values-into-one-column-in-python-pandas
    dados3['Categoria'] = dados3.filter(like="categoria").apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    
    dados3["ano"]=ano
    
    for coluna in dados3.columns:
        if coluna[:10] =="categoria.":
            dados3.drop(columns=[coluna],inplace=True)
    
    return dados3 
    


In [70]:
ano="2019"
var="216"
loc="3"
geo="2"
rel="5457"
ibge = igbe_pam_total(ano=ano,var=var,loc=loc,geo=geo,rel=rel)
ibge

,localidade,valor,variavel,unidade,nome,Categoria,ano
0,Sudeste,15083986,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Total,2019
1,Sudeste,12207,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacate,2019
2,Sudeste,15971,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacaxi,2019
3,Sudeste,51,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Açaí,2019
4,Sudeste,...,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Alfafa fenada,2019
...,...,...,...,...,...,...,...
211,Sudeste,515883,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Trigo (em grão),2019
212,Sudeste,3704,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Triticale (em grão),2019
213,Sudeste,-,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Tungue (fruto seco),2019
214,Sudeste,22438,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Urucum (semente),2019


In [71]:
lista = pd.read_csv("IBGE\Cod reg.csv")
lista.head(3)

,Cod
0,11006
1,11003
2,11005


In [72]:
ibge = igbe_pam_total(ano="2019",var="216",loc="11006",geo="9",rel="5457")
ibge

,localidade,valor,variavel,unidade,nome,Categoria,ano
0,Cacoal (RO),85552,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Total,2019
1,Cacoal (RO),-,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacate,2019
2,Cacoal (RO),155,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Abacaxi,2019
3,Cacoal (RO),5,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Açaí,2019
4,Cacoal (RO),...,Área colhida,Hectares,Produto das lavouras temporárias e permanentes,Alfafa fenada,2019
...,...,...,...,...,...,...,...
211,Cacoal (RO),-,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Trigo (em grão),2019
212,Cacoal (RO),-,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Triticale (em grão),2019
213,Cacoal (RO),-,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Tungue (fruto seco),2019
214,Cacoal (RO),450,Valor da produção,Mil Reais,Produto das lavouras temporárias e permanentes,Urucum (semente),2019


In [73]:
relatorio = pd.DataFrame()
for loc in lista["Cod"]:
    print(str(loc))

11006
11003
11005
11001
11008
11007
11002
11004
12004
12005
12001
12002
12003
13005
13003
13006
13013
13007
13001
13010
13011
13012
13004
13009
13002
13008
14001
14002
14003
14004
15011
15017
15012
15005
15019
15002
15003
15015
15007
15010
15013
15014
15004
15018
15020
15016
15008
15006
15009
15022
15001
15021
16002
16001
16003
16004
17003
17001
17005
17008
17006
17002
17004
17007
21009
21016
21013
21001
21008
21015
21020
21007
21005
21014
21011
21003
21010
21018
21004
21021
21017
21019
21006
21012
21002
22015
22005
22014
22004
22003
22010
22011
22008
22006
22013
22012
22007
22001
22002
22009
23033
23013
23001
23021
23020
23004
23030
23023
23008
23027
23011
23016
23022
23018
23029
23031
23028
23019
23032
23014
23015
23012
23005
23002
23007
23026
23003
23025
23017
23006
23010
23024
23009
24012
24004
24009
24005
24006
24007
24002
24001
24019
24003
24014
24013
24010
24015
24008
24011
24017
24018
24016
25007
25005
25015
25016
25011
25012
25023
25010
25003
25013
25004
25014
25019
25020
2500

In [74]:
relatorio = pd.DataFrame()
for loc in lista["Cod"]:
        relatorio = relatorio.append(igbe_pam_total(ano="2019",var="216",loc=str(loc),geo="9",rel="5457"))
        print(str(loc)," ",relatorio.shape)    

11006   (216, 7)
11003   (432, 7)
11005   (648, 7)
11001   (864, 7)
11008   (1080, 7)
11007   (1296, 7)
11002   (1512, 7)
11004   (1728, 7)
12004   (1944, 7)
12005   (2160, 7)
12001   (2376, 7)
12002   (2592, 7)
12003   (2808, 7)
13005   (3024, 7)
13003   (3240, 7)
13006   (3456, 7)
13013   (3672, 7)
13007   (3888, 7)
13001   (4104, 7)
13010   (4320, 7)
13011   (4536, 7)
13012   (4752, 7)
13004   (4968, 7)
13009   (5184, 7)
13002   (5400, 7)
13008   (5616, 7)
14001   (5832, 7)
14002   (6048, 7)
14003   (6264, 7)
14004   (6480, 7)
15011   (6696, 7)
15017   (6912, 7)
15012   (7128, 7)
15005   (7344, 7)
15019   (7560, 7)
15002   (7776, 7)
15003   (7992, 7)
15015   (8208, 7)
15007   (8424, 7)
15010   (8640, 7)
15013   (8856, 7)
15014   (9072, 7)
15004   (9288, 7)
15018   (9504, 7)
15020   (9720, 7)
15016   (9936, 7)
15008   (10152, 7)
15006   (10368, 7)
15009   (10584, 7)
15022   (10800, 7)
15001   (11016, 7)
15021   (11232, 7)
16002   (11448, 7)
16001   (11664, 7)
16003   (11880, 7)
16004

KeyboardInterrupt: 

In [ ]:
relatorio.to_csv("IBGE.csv",sep=";",decimal=".")
relatorio.head(5)

In [75]:
relatorio = pd.read_csv("IBGE.csv",sep=";",decimal=".")

In [76]:
import pandas as pd

from pivottablejs import pivot_ui

pivot_ui(relatorio)